In [1]:
import os 
import sys 
from dotenv import find_dotenv,load_dotenv
from langchain_groq import  ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS,Chroma
from langchain_core.runnables import RunnablePassthrough


c:\ConferenceBot\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
_ = load_dotenv(find_dotenv())
GROQ_API_KEY = os.getenv('GROQ_API_KEY')
#os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [3]:
# model define 
chat_model = ChatGroq(model="llama-3.3-70b-versatile")

In [4]:
# make vectodatabase
loader = PyMuPDFLoader("../Human_Segmentation.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectordb = FAISS.from_documents(texts, embeddings)
# or use chroma

In [5]:
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k":3})


In [6]:
prompt = ChatPromptTemplate.from_template(
    """
Act as an expert academic reviewer. Analyze the provided research paper and provide a structured review with the following sections:

- **Title Assessment**: Comment on the clarity, relevance, and accuracy of the paper’s title.
- **Abstract Evaluation**: Assess the abstract for conciseness, clarity, and whether it reflects the main content.
- **Strengths**: List the key strengths of the paper, such as novel contributions, robust methodology, or clear writing.
- **Weaknesses**: Identify major weaknesses or areas for improvement, including gaps in methodology, unclear arguments, or missing context.
- **Detailed Comments**: Provide specific, constructive feedback on each section (introduction, methods, results, discussion, etc.), pointing out strengths and areas for improvement.
- **Overall Recommendation**: State your recommendation (accept, minor revision, major revision, reject) and justify your choice based on the above points.

Be precise, structured, and academic in tone. Ground all comments in the paper’s content and avoid speculation. Use bullet points or numbered lists for clarity.

Context:
{context}

Question:
{question}
"""


)

In [7]:
chain   = (
    {
            "context": retriever,
            "question": RunnablePassthrough()
        }
    | prompt
    | chat_model
    
    
)

In [9]:
response = chain.invoke('what are the strengths and weaknesses of the paper?')

In [10]:
print(response.content)

**Title Assessment**: 
The title of the paper is not provided in the given context. However, based on the content, it appears to be related to human segmentation, possibly focusing on the comparison of different encoders and optimizers for U-Net models in the context of semantic segmentation tasks.

**Abstract Evaluation**: 
The abstract is not available in the provided text. However, an effective abstract should concisely summarize the paper's main contributions, methodology, and findings, which seems to be missing here.

**Strengths**:
1. **Comprehensive Comparison**: The paper conducts a thorough comparison of different encoders and optimizers for the U-Net model, providing valuable insights into their performance.
2. **Statistical Analysis**: The use of ANOVA to analyze the statistical significance of differences between optimizers is a strength, demonstrating a rigorous approach to evaluating performance metrics.
3. **Clear Presentation of Results**: The inclusion of tables (like 